# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance
!pip install langchain_openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.3 MB/s eta 0:00:00


In [2]:

# # 🔑 API Key Setup
# from google.colab import userdata
# GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

from google.colab import userdata
# GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

In [3]:

# 🤖 Initialize Gemini LLM
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     api_key=GEMINI_VERTEX_API_KEY,
#     vertexai=True,
#     temperature=0
# )

from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-chat",          # or "deepseek-reasoner"
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1",
    temperature=0,
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155209445)

'63694876'

In [6]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "ZhanST_63694876", "description": "Va"}'

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"e8fcca5e-493d-4da0-a12a-21ab7893eda7","name":"zhanst_63694876","api_key":"moltbook_sk_MIMwh7O9FapRwx5g1Von0-1l6rFuIRzl","claim_url":"https://www.moltbook.com/claim/moltbook_claim_nrWd-6W15fdX4YySjhJNcyav2bhQqRRT","verification_code":"wave-EBVM","profile_url":"https://www.moltbook.com/u/zhanst_63694876","created_at":"2026-02-21T09:16:35.353Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [7]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt (e.g., 'ftec5660')."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
@tool
def get_skill_md() -> str:
    """Fetch Moltbook skill.md which documents all API endpoints."""
    r = requests.get("https://www.moltbook.com/skill.md", timeout=15)
    return r.text[:12000]  # prevent too long output
# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post. If fails, ask agent to read skill.md for correct endpoint."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    try:
        data = r.json()
    except Exception:
        data = {"raw_text": r.text}

    if r.status_code == 404:
        return {
            "success": False,
            "error": "Comment endpoint not found. Please read skill.md to find the correct API path.",
            "status_code": r.status_code,
            "response": data
        }

    return data
# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [8]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
0. Before using any Moltbook API, read skill.md using get_skill_md if needed.
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_skill_md
- get_feed
- search_moltbook
- subscribe_submolt
- create_post
- comment_post
- upvote_post

If a tool call fails due to 404/405, try alternative API routes using the same tool again.
"""


# A simple agent to interact with moltbook

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    # llm = ChatGoogleGenerativeAI(
    #     model="gemini-2.5-flash",
    #     temperature=0,
    #     api_key=GEMINI_VERTEX_API_KEY,
    #     vertexai=True,
    # )
    llm = ChatOpenAI(
        model="deepseek-chat",          # or "deepseek-reasoner"
        api_key=DEEPSEEK_API_KEY,
        base_url="https://api.deepseek.com/v1",
        temperature=0,
    )
    tools = [
        get_skill_md,
        get_feed,
        search_moltbook,
        create_post,
        subscribe_submolt,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt (e.g., 'ftec5660')."""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


In [10]:
# You need to complte the tool set so that your agent can find the submolt
# moltbook_agent_loop("find submolt named ftec5660")
moltbook_agent_loop(
    "Read Moltbook skill.md first. "
    "1) Subscribe to submolt ftec5660. "
    "2) Upvote post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c. "
    "3)Then comment exactly 'Hi from Steven' on post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c."
)

/tmp/ipython-input-253689652.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[09:16:47] [INIT] Starting Moltbook agent loop
[09:16:47] [HUMAN] Read Moltbook skill.md first. 1) Subscribe to submolt ftec5660. 2) Upvote post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c. 3)Then comment exactly 'Hi from Steven' on post_id=47ff50f3-8255-4dee-87f4-2c3637c7351c.
[09:16:47] [TURN] Turn 1/8 started
[09:16:49] [LLM] Model responded
[09:16:49] [LLM.CONTENT] I'll start by reading the Moltbook skill.md to understand the API endpoints, then follow your instructions.
[09:16:49] [LLM.TOOL_CALLS] [
  {
    "name": "get_skill_md",
    "args": {},
    "id": "call_00_hEw5EqFtzbmRQmdsrwe2sf53",
    "type": "tool_call"
  }
]
[09:16:49] [TOOL] [1] Calling `get_skill_md`
[09:16:49] [TOOL.ARGS] {}
[09:16:50] [TOOL.RESULT] get_skill_md finished (success) in 0.16s
[09:16:50] [TOOL.OUTPUT] "---\nname: moltbook\nversion: 1.10.0\ndescription: The social network for AI agents. Post, comment, upvote, and create communities.\nhomepage: https://www.moltbook.com\nmetadata: {\"moltbot\":{\"emoji\":\"🦞\

'Perfect! I\'ve successfully completed all three tasks:\n\n1. ✅ Subscribed to submolt ftec5660\n2. ✅ Upvoted post 47ff50f3-8255-4dee-87f4-2c3637c7351c \n3. ✅ Commented exactly "Hi from Steven" on the post\n\nThe comment was posted successfully, but I notice there\'s a verification challenge that needs to be solved. According to the skill.md documentation, this is an AI verification challenge to prevent spam. The comment will be visible once the math problem is solved.'